In [ ]:
# Este código está pensado para correr en un Jupyter Notebook, puede usar de manera gratuita un laboratorio pynb de Google Colab: https://colab.research.google.com/?hl=es
# Documentación completa: *en desarrollo*
# Github: https://github.com/JuanCAPlos/ChatGPT-ElevenLabs_HumanIA.git

!pip install -q openai
!pip install -q elevenlabs

import os
import openai
import tempfile
import requests
from IPython.display import Audio, clear_output
from elevenlabs import generate, play, set_api_key, voices, Models

#APIs
openai_api_key     = "YOUR_OPEN_AI_API_KEY" 
eleven_api_key = "YOUR_ELEVENLABS_API_KEY" 

# Configura las APIs de OpenAI y de ElevenLabs
openai.api_key = openai_api_key
set_api_key(eleven_api_key)

voice_list = voices()

import ipywidgets as widgets

voice_labels = [voice.category + " voice: " + voice.name for voice in voice_list]

voice_id_dropdown = widgets.Dropdown(
    options=voice_labels,
    value=voice_labels[0],
    description="Selecciona una voz:",
)

display(voice_id_dropdown)

#Configuración de ChatGPT-3
chatgpt_model = "gpt-3.5-turbo"

chatgpt_system = "You are a helpful assistant on a conversation. Answer should be not too long. Be ironic and acid"

#Encuentra el índice de la opción seleccionada
selected_voice_index = voice_labels.index(voice_id_dropdown.value)
selected_voice_id    = voice_list[selected_voice_index].voice_id

#Función para obtener la respuesta de ChatGPT
def get_gpt4_response(prompt):
    response = openai.ChatCompletion.create(
        model=chatgpt_model,
        messages=[
            {"role": "system", "content": chatgpt_system},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

#Función principal para interactuar con ChatGPT
def interact_with_gpt4(prompt):
    response_text = get_gpt4_response(prompt)

    import requests

    CHUNK_SIZE = 1024
    url = "https://api.elevenlabs.io/v1/text-to-speech/oF718J255UdCqJHYZZcj"

    headers = {
      "Accept": "audio/mpeg",
      "Content-Type": "application/json",
      "xi-api-key": eleven_api_key
    }

    data = {
      "text": response_text,
      "model_id" : "eleven_multilingual_v1",
      "voice_settings": {
        "stability": 0.4,
        "similarity_boost": 1.0
      }
    }

    response = requests.post(url, json=data, headers=headers)
    
    #Guardar datos de audio en un archivo temporal
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
        for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
            if chunk:
                f.write(chunk)
        f.flush()
        temp_filename = f.name

    return temp_filename

#Función de interacción constante con ChatGPT
def continuous_interaction():
    while True:
        clear_output(wait=True)
        prompt = input("Enter your prompt (or type 'exit' to stop): ")
        if prompt.lower() == 'exit':
            break
        audio_file = interact_with_gpt4(prompt)
        play(audio_file, notebook=True)

#Ejemplo de uso
continuous_interaction()